<a href="https://colab.research.google.com/github/SachinSouravJha/Machine-Learning-Basics/blob/master/SepsisWithXgBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://www.dropbox.com/s/xdqcrhlh5rwrvzm/sepsis.csv?dl=0

--2020-02-26 20:38:31--  https://www.dropbox.com/s/xdqcrhlh5rwrvzm/sepsis.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xdqcrhlh5rwrvzm/sepsis.csv [following]
--2020-02-26 20:38:31--  https://www.dropbox.com/s/raw/xdqcrhlh5rwrvzm/sepsis.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc75332a72ef0f42dcce6f7f98ff.dl.dropboxusercontent.com/cd/0/inline/Ay3xXqJKEfYejSek70xKNp4RRhxiqqxiX3ITrj97bmPXNuPCR0mlNAf4KpkFuTpqarC9BexpGC_qXsu1QpIvJvkuHwdTBP3csAGif7lxLTBYx-YJgvqcJLoCqGIjj14IUak/file# [following]
--2020-02-26 20:38:32--  https://uc75332a72ef0f42dcce6f7f98ff.dl.dropboxusercontent.com/cd/0/inline/Ay3xXqJKEfYejSek70xKNp4RRhxiqqxiX3ITrj97bmPXNuPCR0mlNAf4KpkFuTpqarC9BexpGC_qXsu1QpIvJvkuHwdTBP3csAGif7lxLTBYx-YJgvq

In [0]:
!ls

 sample_data  'sepsis.csv?dl=0'


In [0]:
import pandas as pd
sepsis_data = pd.read_csv('sepsis.csv?dl=0', sep ='|')

In [0]:
print(sepsis_data.head())

      HR  O2Sat  Temp    SBP  ...  Unit2  HospAdmTime  ICULOS  SepsisLabel
0    NaN    NaN   NaN    NaN  ...    NaN        -0.03       1            0
1   97.0   95.0   NaN   98.0  ...    NaN        -0.03       2            0
2   89.0   99.0   NaN  122.0  ...    NaN        -0.03       3            0
3   90.0   95.0   NaN    NaN  ...    NaN        -0.03       4            0
4  103.0   88.5   NaN  122.0  ...    NaN        -0.03       5            0

[5 rows x 41 columns]


In [0]:
sepsis_data.shape

(1361672, 41)

In [0]:
#We have a lot of columns with a lot of missing values so we just need to remove those columns 

In [0]:
# Sum of all missing values in dataset
missing = sepsis_data.isnull().sum() #Using this we will get the number of rows in which the value is null for that column 
missing

HR                   130133
O2Sat                175676
Temp                 902590
SBP                  199680
MAP                  165134
DBP                  459960
Resp                 198043
EtCO2               1319201
BaseExcess          1277998
HCO3                1297001
FiO2                1236770
pH                  1258514
PaCO2               1279813
SaO2                1311871
AST                 1339842
BUN                 1265550
Alkalinephos        1340093
Calcium             1283397
Chloride            1292412
Creatinine          1277512
Bilirubin_direct    1359132
Glucose             1138773
Lactate             1323998
Magnesium           1272850
Phosphate           1304990
Potassium           1231945
Bilirubin_total     1341905
TroponinI           1350388
Hct                 1235350
Hgb                 1258251
PTT                 1317887
WBC                 1272210
Fibrinogen          1352505
Platelets           1279609
Age                       0
Gender              

In [0]:
#We can see that there are rows in which the numbe of missing values is a lot, so let's take percentage and then we can just remove columns with more than 90% values
missing_percent = (missing/1361672)*100 #Missing in all columns/Number of rows in each column 
missing_percent

HR                   9.556854
O2Sat               12.901492
Temp                66.285420
SBP                 14.664324
MAP                 12.127296
DBP                 33.779060
Resp                14.544105
EtCO2               96.880967
BaseExcess          93.855055
HCO3                95.250618
FiO2                90.827306
pH                  92.424167
PaCO2               93.988347
SaO2                96.342658
AST                 98.396824
BUN                 92.940884
Alkalinephos        98.415257
Calcium             94.251553
Chloride            94.913606
Creatinine          93.819363
Bilirubin_direct    99.813465
Glucose             83.630493
Lactate             97.233254
Magnesium           93.476990
Phosphate           95.837324
Potassium           90.472963
Bilirubin_total     98.548329
TroponinI           99.171313
Hct                 90.723023
Hgb                 92.404852
PTT                 96.784468
WBC                 93.429989
Fibrinogen          99.326784
Platelets 

In [0]:
#Taking the list the of all columns which has more than 90% missing values
drop_cols = list(missing_percent[missing_percent > 90].index)
drop_cols

['EtCO2',
 'BaseExcess',
 'HCO3',
 'FiO2',
 'pH',
 'PaCO2',
 'SaO2',
 'AST',
 'BUN',
 'Alkalinephos',
 'Calcium',
 'Chloride',
 'Creatinine',
 'Bilirubin_direct',
 'Lactate',
 'Magnesium',
 'Phosphate',
 'Potassium',
 'Bilirubin_total',
 'TroponinI',
 'Hct',
 'Hgb',
 'PTT',
 'WBC',
 'Fibrinogen',
 'Platelets']

In [0]:
sepsis_data = sepsis_data.drop(drop_cols,axis=1) #Removes those columns
missing_data = sepsis_data.isnull().sum() #Checking missed percent in the columns which are left
missing_percent = (missing_data/sepsis_data.shape[0])*100 #Calculating the percentage 
missing_percent

HR              9.556854
O2Sat          12.901492
Temp           66.285420
SBP            14.664324
MAP            12.127296
DBP            33.779060
Resp           14.544105
Glucose        83.630493
Age             0.000000
Gender          0.000000
Unit1          40.778029
Unit2          40.778029
HospAdmTime     0.000588
ICULOS          0.000000
SepsisLabel     0.000000
dtype: float64

In [0]:
sepsis_data.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,83.14,0,NaN,NaN,-0.03,2,0
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,83.14,0,NaN,NaN,-0.03,3,0
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,83.14,0,NaN,NaN,-0.03,4,0
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,83.14,0,NaN,NaN,-0.03,5,0


In [0]:
sepsis_data.shape

(1361672, 15)

In [0]:
#We are now left with 15 columns instead of 41 we had at start, in this the HospAdmTime does not look useful so I will drop this too
sepsis_data = sepsis_data.drop(["HospAdmTime"],axis=1)

In [0]:
sepsis_data.head()

In [0]:
sepsis_data["HR"] = sepsis_data["HR"].fillna(80)
#I googled to find heart rate of a normal person and filled that for missing values
#If I am a doctor and I don't have report, I will assume it to be normal so it makes sense

In [0]:
sepsis_data.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,Unit1,Unit2,ICULOS,SepsisLabel
0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,1,0
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,83.14,0,NaN,NaN,2,0
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,83.14,0,NaN,NaN,3,0
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,83.14,0,NaN,NaN,4,0
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,83.14,0,NaN,NaN,5,0


In [0]:
sepsis_data = sepsis_data.drop(["Unit1","Unit2"],axis=1)
sepsis_data.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,ICULOS,SepsisLabel
0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,1,0
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,83.14,0,2,0
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,83.14,0,3,0
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,83.14,0,4,0
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,83.14,0,5,0


In [0]:
sepsis_data["O2Sat"] = sepsis_data["O2Sat"].fillna(97)
sepsis_data["Temp"] = sepsis_data["Temp"].fillna(37.2)
sepsis_data["SBP"] = sepsis_data["SBP"].fillna(140)
sepsis_data["MAP"] = sepsis_data["MAP"].fillna(85)
sepsis_data["DBP"] = sepsis_data["DBP"].fillna(90)
sepsis_data["Glucose"] = sepsis_data["Glucose"].fillna(112)
sepsis_data["Resp"] = sepsis_data["Resp"].fillna(20)
sepsis_data["Age"] = sepsis_data["Age"].fillna(sepsis_data["Age"].mean())

In [0]:
sepsis_data.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Glucose,Age,Gender,ICULOS,SepsisLabel
0,80.0,97.0,37.2,140.0,85.00,90.0,20.0,112.0,83.14,0,1,0
1,97.0,95.0,37.2,98.0,75.33,90.0,19.0,112.0,83.14,0,2,0
2,89.0,99.0,37.2,122.0,86.00,90.0,22.0,112.0,83.14,0,3,0
3,90.0,95.0,37.2,140.0,85.00,90.0,30.0,112.0,83.14,0,4,0
4,103.0,88.5,37.2,122.0,91.33,90.0,24.5,112.0,83.14,0,5,0


In [0]:
from sklearn.preprocessing import MinMaxScaler #Normalizing the data
scaler = MinMaxScaler() 
data = scaler.fit_transform(sepsis_data)
data[1]

array([0.29615385, 0.9375    , 0.56013746, 0.27857143, 0.19760714,
       0.25      , 0.18181818, 0.10429448, 0.80395349, 0.        ,
       0.00298507, 0.        ])

In [0]:
def hypothesis(x,w,b):
    '''accepts input vector x, input weight vector w and bias b'''
    
    h = np.dot(x,w) + b #Doing dot ensures we don't need to use loop to multiply both variables with their respective weights
    return sigmoid(h)
    #Calculating the score on the basis of weight and variable's value + bias
    #Then simply put that value in sigmoid to get a value between 0 and 1
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-1.0*z))

def error(y_true,x,w,b):
    
    m = x.shape[0]
    
    err = 0.0
    
    for i in range(m):
        hx = hypothesis(x[i],w,b) 
        err += y_true[i]*np.log2(hx) + (1-y_true[i])*np.log2(1-hx) #This is simply LL function which we need to maximize
        
    
    return -err/m #Negative of LL


def get_grads(y_true,x,w,b):
    
    grad_w = np.zeros(w.shape)
    grad_b = 0.0
    
    m = x.shape[0]
    
    for i in range(m):
        hx = hypothesis(x[i],w,b)
        
        grad_w += (y_true[i] - hx)*x[i]
        grad_b +=  (y_true[i]-hx) #Basically multiplies by x[0] which is 1
        
    
    grad_w /= m
    grad_b /= m #Average Error and Average Gradient
    
    return [grad_w,grad_b]


# One Iteration of Gradient Descent
def grad_descent(x,y_true,w,b,learning_rate=0.1):
    loss = []
    for i in range(100):
        err = error(y_true,x,w,b)
        loss.append(err)
        [grad_w,grad_b] = get_grads(y_true,x,w,b)
    
        w = w + learning_rate*grad_w
        b = b + learning_rate*grad_b
    #So after applying gradient descent our weights and biases are updating themselves 
    return loss,w,b

In [0]:
def predict(x,w,b):
    
    confidence = hypothesis(x,w,b)
    if confidence<0.5:
        return 0
    else:
        return 1
    
def get_acc(x_tst,y_tst,w,b):
    
    y_pred = []
    
    for i in range(y_tst.shape[0]):
        p = predict(x_tst[i],w,b)
        y_pred.append(p)
        
    y_pred = np.array(y_pred)
    return  float((y_pred==y_tst).sum())/y_tst.shape[0]

In [0]:
split = int(0.8*data.shape[0]) #We will include 80% in training set while rest will go to test
X_train = data[:split,:-1]
X_test = data[split:,:-1]

Y_train = data[:split,-1]
Y_test  = data[split:,-1]

print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(1089337, 11) (272335, 11)
(1089337,) (272335,)


In [0]:
w = 2*np.random.random((X_train.shape[1],))#Taking same number of random variables as weights
b = 5*np.random.random()#Taking a random bias
loss = []
loss,w,b = grad_descent(X_train,Y_train,w,b,learning_rate=0.1)
acc = get_acc(X_test,Y_test,w,b)
print(loss)
print(acc)

[5.585596897496662, 5.159118646594576, 4.7392077126609555, 4.327922254770561, 3.9278098016078724, 3.541910152849486, 3.1736753383982563, 2.826771368346263, 2.5047478818255025, 2.2106091939540113, 1.9463806031591309, 1.712800696323262, 1.5092457541091189, 1.3339065203966172, 1.1841412936552522, 1.0568818227613541, 0.9489861478156832, 0.8574855110635481, 0.7797220228100208, 0.713401001981093, 0.6565883810283469, 0.6076787228907806, 0.5653513031741856, 0.5285245121192953, 0.49631375941389094, 0.4679949626331867, 0.4429740069222835, 0.4207617489911826, 0.40095382100063665, 0.3832144295408201, 0.36726340338936236, 0.35286584690650746, 0.3398238668478217, 0.32796994254685635, 0.3171615968820188, 0.3072770973585685, 0.29821197438344726, 0.2898761895218037, 0.2821918224092104, 0.27509117303255776, 0.26851519796217904, 0.2624122161693667, 0.2567368333746165, 0.25144904428259046, 0.2465134802250155, 0.2418987761539599, 0.23757703599771993, 0.2335233794057835, 0.22971555610181577, 0.2261336166118

In [0]:
print(acc*100) #For percentage
print(classification_report(y_valid, xgb_prediction))

98.62669139111756


In [0]:
sepsis_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361672 entries, 0 to 1361671
Data columns (total 13 columns):
HR             1361672 non-null float64
O2Sat          1361672 non-null float64
Temp           1361672 non-null float64
SBP            1361672 non-null float64
MAP            1361672 non-null float64
DBP            1361672 non-null float64
Resp           1361672 non-null float64
Glucose        1361672 non-null float64
Age            1361672 non-null float64
Gender         1361672 non-null int64
HospAdmTime    1361664 non-null float64
ICULOS         1361672 non-null int64
SepsisLabel    1361672 non-null int64
dtypes: float64(10), int64(3)
memory usage: 135.1 MB


In [0]:
X = data[:,:-1]
Y = data[:,-1]
print(X.shape)
print(Y.shape)

(1361672, 11)
(1361672,)


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X,Y, test_size = 0.15, random_state = 42)

In [0]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)
y_pred = LR.predict(X_valid)
acc= 'Accuracy:{:.3f}'.format(LR.score(X_valid,y_valid))
print(acc)

Accuracy:0.982


In [0]:
#To be continued in future

In [0]:
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2)) #We will find squares of distance between the two matrices
    #Then we will sum the squares
def knn(X,Y,queryPoint,k=5):
    #Writing KNN
    vals = []
    m = X.shape[0]
    #Traversing all rows
    for i in range(m):
        d = dist(queryPoint,X[i])
        #Here we will find 
        vals.append((d,Y[i]))
        #Updating array with the distance and it's value
    
    vals = sorted(vals)
    # Nearest/First K points
    vals = vals[:k]
    
    vals = np.array(vals)
    #Converting into numpy
    #print(vals)
    
    new_vals = np.unique(vals[:,1],return_counts=True) #This array will store all the unique values in the second column 
    #Then it will also frequency of the numbers
    
    #print(new_vals)
    
    index = new_vals[1].argmax()
    #Here we found the index of number which came most number of times
    pred = new_vals[0][index]
    #Using that index we find the number which came most number of times
    return pred

In [0]:
def get_acc(X_train1,Y_train1,X_test1,Y_test1):
    
    y_pred = []
    
    for i in range(Y_test.shape[0]):
        p = knn(X_train,Y_train,X_test[1])
        y_pred.append(p)
        
    y_pred = np.array(y_pred)
    
    return float((y_pred==Y_test).sum())/Y_test.shape[0]

In [0]:
X1 = data[:50000,1:]
Y1 = data[:50000,0]
#All the variables go in X
#The answers to each sets of variables is in Y
print(X1.shape,Y1.shape)

(50000, 11) (50000,)


In [0]:
split = int(0.8*X1.shape[0])
print(split)
#Not using libraries
#Causing a split to divide into test and training set
X_train1 = X1[:split,:]
Y_train1 = Y1[:split]

X_test1 = X1[split:,:]
Y_test1 = Y1[split:]

print(X_train1.shape,Y_train1.shape)
print(X_test1.shape,Y_test1.shape)

40000
(40000, 11) (40000,)
(10000, 11) (10000,)


In [0]:
accuracy = get_acc(X_train1,Y_train1,X_test1,Y_test1)
print(accuracy*100)

In [0]:
def prior_prob(y_train,label):
    total = y_train.shape[0]
    ClassLabel = np.sum(y_train==label) #It will compute number of datasets equal to reuired label in whole dataset
    return (ClassLabel/total)

In [0]:
def cond_prob(x_train,y_train,feature_col,feature_val,label):
    
    x_filter = x_train[y_train==label] #We will just consider rows where labels as we want them to be
    numerator = np.sum(x_filter[:,feature_col]==feature_val) #We will get the count of the required labels among the
    #correct labels
    denominator = np.sum(y_train==label)#Count of rows with correct labels
    
    return numerator/float(denominator)

In [0]:
#We will get likelihood of all the labels
#To be a probability of a certain label if given a certain feature we can use our above function
#Then we can keep multiplying for all the features and get the likelihood
#Then we can do this for all the labels

def predict(x_train,y_train,x_test):
    """Xtest is a single testing point, n features"""
    
    classes = np.unique(y_train) #Number of unique classes
    n_features = x_train.shape[1] #Number of features
    probabilities = [] # List of prob for all classes and given a single testing point
    #Compute Posterior for each class
    for label in classes: #For each label
        
        #Post_c = likelihood*prior
        likelihood = 1.0
        for f in range(n_features): #Checks all features
            cond = cond_prob(x_train,y_train,f,x_test[f],label)
            likelihood *= cond 
            
        prior = prior_prob(y_train,label)
        post = likelihood*prior
        probabilities.append(post)
        
    pred = np.argmax(probabilities) #Take the maximum
    return pred

In [0]:
X1 = data[:400000,:-1]
Y1 = data[:400000,-1]
print(X1.shape)
print(Y1.shape)

(400000, 11)
(400000,)


In [0]:
from sklearn.model_selection import train_test_split #Could have done ourselves too but it saves the work
X_train,X_valid,y_train,y_valid = train_test_split(X1,Y1, test_size = 0.2, random_state = 42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(320000, 11)
(80000, 11)
(320000,)
(80000,)


In [0]:

import xgboost as xgb

xgb_model = xgb.XGBClassifier(max_depth=150, n_estimators=100, objective='binary:logistic').fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)


In [0]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print('training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('testing score:', f1_score(y_valid, xgb_prediction, average='weighted'))
print(classification_report(y_valid, xgb_prediction))

training score: 0.995774104449618
testing score: 0.9800932331686867
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     78321
         1.0       0.93      0.27      0.42      1679

    accuracy                           0.98     80000
   macro avg       0.96      0.64      0.71     80000
weighted avg       0.98      0.98      0.98     80000



In [0]:
import numpy as np
def accuarca(x_train,y_train,x_test,y_test):

    pred = []   #making a prediction array
    for i in range(x_test.shape[0]):
        pred_label = predict(x_train,y_train,x_test[i]) #Predicting for each test
        pred.append(pred_label) # <===Correction
    
    pred = np.array(pred) #Converting it to array
    
    accuracy = np.sum(pred==y_test)/y_test.shape[0] #Dividing all correct predictions/ Total number of tests
    return accuracy

print(accuarca(x_train,y_train,x_test,y_test))